In [18]:
from pyscipopt import Model

m = Model("demo")
x = m.addVar("x", lb=0, ub=10)
m.setObjective(x, "maximize")
m.addCons(x <= 5)
m.optimize()
print("Optimal value:", m.getObjVal())

Optimal value: 5.0
feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 1 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +5.00000000000000e+00 (2 solutions)
Dual Bound         : +5.00000000000000e+00
Gap                : 0.00 %


In [19]:
from pyscipopt import Model, quicksum
import numpy as np

# Number of cities
n = 5

# Generate a random symmetric distance matrix
np.random.seed(42)
dist = np.random.randint(10, 100, size=(n, n))
for i in range(n):
    dist[i, i] = 0
    for j in range(i+1, n):
        dist[j, i] = dist[i, j]

print("Distance matrix:\n", dist)

# Create SCIP model
model = Model("TSP_MTZ")

# Binary variables: x[i,j] = 1 if path goes from i -> j
x = {}
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

# MTZ ordering variables: u[i] gives position of city i in tour
u = {}
for i in range(n):
    u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

# Objective: minimize travel distance
model.setObjective(
    quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
    "minimize"
)

# Each city has exactly one outgoing edge
for i in range(n):
    model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)

# Each city has exactly one incoming edge
for j in range(n):
    model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

# MTZ subtour elimination constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

# Solve the model
model.optimize()

# Print optimal solution
if model.getStatus() == "optimal":
    print(f"\nOptimal tour length = {model.getObjVal():.2f}")

    # Extract tour edges
    tour = []
    current = 0
    visited = {0}
    for _ in range(n):
        for j in range(n):
            if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                tour.append((current, j))
                current = j
                break
    print("Tour edges:", tour)


Distance matrix:
 [[ 0 24 81 70 30]
 [24  0 84 84 97]
 [81 84  0 62 11]
 [70 84 62  0 73]
 [30 97 11 73  0]]

Optimal tour length = 211.00
Tour edges: [(0, 1), (1, 3), (3, 2), (2, 4), (4, 0)]
presolving:
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 10 clqs
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 10 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 24 implications, 10 cliques
presolved problem has 24 variables (20 bin, 0 int, 0 impl, 4 cont) and 22 constraints
     10 constrai

In [20]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import random

def solve_tsp_mtz(n, dist, instance_id):
    """
    Solve TSP with MTZ constraints using SCIP.
    Returns objective value and edges in the optimal tour.
    """
    model = Model(f"TSP_MTZ_{instance_id}")
    
    # Decision variables
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[i, j] = model.addVar(vtype="B", name=f"x({i},{j})")

    u = {}
    for i in range(n):
        u[i] = model.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})")

    # Objective
    model.setObjective(
        quicksum(dist[i, j] * x[i, j] for i in range(n) for j in range(n) if i != j),
        "minimize"
    )

    # Degree constraints
    for i in range(n):
        model.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        model.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    # MTZ constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addCons(u[i] - u[j] + n * x[i, j] <= n - 1)

    # Solve
    model.optimize()

    if model.getStatus() == "optimal":
        obj = model.getObjVal()
        # Extract tour edges
        edges = []
        current = 0
        visited = {0}
        for _ in range(n):
            for j in range(n):
                if current != j and (current, j) in x and model.getVal(x[current, j]) > 0.5:
                    edges.append((current, j))
                    current = j
                    break
        return obj
    else:
        return None, None


def generate_instances(num_instances=100, min_cities=5, max_cities=10, seed=42):
    """
    Generate and solve multiple random TSP instances.
    Returns a pandas DataFrame with summary results.
    """
    random.seed(seed)
    np.random.seed(seed)

    results = []
    for inst in range(num_instances):
        n = random.randint(min_cities, max_cities)
        dist = np.random.randint(10, 100, size=(n, n))
        for i in range(n):
            dist[i, i] = 0
            for j in range(i+1, n):
                dist[j, i] = dist[i, j]

        obj = solve_tsp_mtz(n, dist, inst)
        if obj is not None:
            results.append({
                "instance": inst,
                "cities": n,
                "objective": obj
            })
            print(f"Instance {inst}: {n} cities, optimal cost = {obj:.2f}")
        else:
            print(f"Instance {inst}: solver failed")

    return pd.DataFrame(results)


# Example: generate 10 small TSP instances
df = generate_instances(num_instances=10, min_cities=5, max_cities=7)

print(df.head())


Instance 0: 7 cities, optimal cost = 261.00presolving:
(round 1, fast)       1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 14 clqs
(round 2, exhaustive) 1 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 14 upgd conss, 0 impls, 14 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 1 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 60 implications, 14 cliques
presolved problem has 48 variables (42 bin, 0 int, 0 impl, 6 cont) and 44 constraints
     14 constraints of type <setppc>
     30 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | no

In [52]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE
import math

INT_TOL = 1e-6  # integrality tolerance

def _is_integral(model, vars_):
    """Check if current LP solution (at focused node) is integral on all integer vars."""
    for v in vars_:
        if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
            val = model.getSolVal(None, v)  # None = current LP solution
            if abs(val - round(val)) > INT_TOL:
                return False
    return True

class NodeTracker(Eventhdlr):
    """
    Version-safe tracker for PySCIPOpt 5.6.0:
    - subscribes only to events your build exposes
    - READ-ONLY in callbacks (no parameter changes)
    - logs integer-feasible solutions and lightweight node info
    """
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.active = {}     # node_id -> dict(status, depth, lp_bound, integral)
        self.sols = []       # [{'obj': float|None, 'values': {var: int}}]
        self.allvars = None

    def eventinit(self):
        # Use only events you actually have
        for et in [
            SCIP_EVENTTYPE.NODEBRANCHED,
            SCIP_EVENTTYPE.NODEFOCUSED,
            SCIP_EVENTTYPE.LPSOLVED,
            SCIP_EVENTTYPE.NODEINFEASIBLE,
            SCIP_EVENTTYPE.NODEFEASIBLE,
            SCIP_EVENTTYPE.NODESOLVED,
            SCIP_EVENTTYPE.BESTSOLFOUND,
        ]:
            try:
                self.model.catchEvent(et, self)
            except Exception:
                pass
        self.allvars = self.model.getVars()

    def _mark_open(self, node):
        try:
            nid = node.getNumber()
            depth = self.model.getDepth()
        except Exception:
            return
        if nid not in self.active:
            self.active[nid] = {
                "status": "open",
                "depth": depth,
                "lp_bound": None,
                "integral": None,
            }

    def _close(self, node, reason):
        try:
            nid = node.getNumber()
        except Exception:
            return
        if nid in self.active:
            self.active[nid]["status"] = reason

    def eventexec(self, event):
        et = event.getType()

        if et == SCIP_EVENTTYPE.NODEBRANCHED:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node:
                self._mark_open(node)

        elif et == SCIP_EVENTTYPE.NODEFOCUSED:
            node = self.model.getCurrentNode()
            if node:
                self._mark_open(node)

        elif et == SCIP_EVENTTYPE.LPSOLVED:
            node = self.model.getCurrentNode()
            if not node:
                return
            self._mark_open(node)

            # LP bound (may fail for some nodes; guard)
            try:
                lb = self.model.getLPObjVal()
            except Exception:
                lb = None
            nid = node.getNumber()
            self.active[nid]["lp_bound"] = lb

            # integrality on current LP sol
            integral = _is_integral(self.model, self.allvars)
            self.active[nid]["integral"] = integral
            if integral:
                # Log solution (read-only)
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        try:
                            vals[v.name] = int(round(self.model.getSolVal(None, v)))
                        except Exception:
                            pass
                try:
                    obj = self.model.getLPObjVal()
                except Exception:
                    obj = None
                self.sols.append({"obj": obj, "values": vals})
                self._close(node, "integer")

        elif et == SCIP_EVENTTYPE.NODEINFEASIBLE:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node:
                self._close(node, "infeasible")

        elif et in (SCIP_EVENTTYPE.NODEFEASIBLE, SCIP_EVENTTYPE.NODESOLVED):
            # Optional: could mark node as solved if it's still open
            node = self.model.getCurrentNode()
            if node and self.active.get(node.getNumber(), {}).get("status") == "open":
                self._close(node, "solved")

        elif et == SCIP_EVENTTYPE.BESTSOLFOUND:
            # Also log global solutions (from heuristics, etc.)
            sol = self.model.getBestSol()
            if sol:
                try:
                    obj = self.model.getSolObjVal(sol)
                except Exception:
                    obj = None
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        try:
                            vals[v.name] = int(round(self.model.getSolVal(sol, v)))
                        except Exception:
                            pass
                self.sols.append({"obj": obj, "values": vals})

In [53]:
def _safe_set_param(model, name, value):
    try:
        if hasattr(model, "hasParam") and model.hasParam(name):
            if isinstance(value, (bool, int)):
                model.setIntParam(name, int(value)); return
            if isinstance(value, float):
                model.setRealParam(name, float(value)); return
            if isinstance(value, str):
                model.setStringParam(name, value); return
        # fallback probing
        try: model.setIntParam(name, int(value)); return
        except: pass
        try: model.setRealParam(name, float(value)); return
        except: pass
        try: model.setStringParam(name, str(value)); return
        except: pass
    except:
        pass


In [54]:
def solve_ip_enumerate_like(build_model_fn, node_limit, time_limit=None,
                            damp_root=True):
    model = build_model_fn()

    # Make it less likely to close at root (only if these params exist)
    if damp_root:
        _safe_set_param(model, "presolving/maxrounds", 0)
        _safe_set_param(model, "separating/maxrounds", 0)
        _safe_set_param(model, "separating/maxstallrounds", 0)
        _safe_set_param(model, "heuristics/freq", -1)      # disable many heuristics
        _safe_set_param(model, "heuristics/emphasis", 0)   # if present
        _safe_set_param(model, "display/verblevel", 0)     # optional quieter logs

    # Stop before optimality
    model.setLongintParam("limits/nodes", int(node_limit))
    if time_limit is not None:
        model.setRealParam("limits/time", float(time_limit))

    # Attach your read-only NodeTracker (as you already have)
    tracker = NodeTracker()
    model.includeEventhdlr(tracker, "NodeTracker", "Log integer nodes")

    model.optimize()

    # Active nodes directly from SCIP
    leaves, children, siblings = model.getOpenNodes()
    def node_info(n):
        try: lb = n.getLowerbound()
        except: lb = None
        return {
            "node_id": n.getNumber(),
            "parent_id": (n.getParent().getNumber() if n.getParent() else None),
            "depth": n.getDepth(),
            "lower_bound": lb,
            "is_active": n.isActive(),
        }
    active_nodes = [node_info(n) for n in (leaves + children + siblings)]

    # Node counts (5.6.0 name)
    try: nodes_remaining = model.getNNodesLeft()
    except AttributeError: nodes_remaining = None

    out = {
        "status": model.getStatus(),
        "nodes_processed": model.getNNodes(),
        "nodes_remaining": nodes_remaining,
        "solving_time": model.getSolvingTime(),
        "integer_solutions": tracker.sols,   # or dedup if you also add model.getSols()
        "active_nodes": active_nodes,
    }
    try: model.free()
    except: pass
    return out


In [55]:
def build_tsp_mtz_euclid(n=15, seed=1):
    rng = np.random.default_rng(seed)
    pts = rng.random((n, 2))
    dist = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i == j: continue
            d = np.linalg.norm(pts[i] - pts[j])
            dist[i, j] = int(round(100 * d))  # rounded Euclidean

    m = Model("TSP_MTZ")
    x = {(i, j): m.addVar(vtype="B", name=f"x({i},{j})")
         for i in range(n) for j in range(n) if i != j}
    u = {i: m.addVar(lb=0, ub=n-1, vtype="C", name=f"u({i})") for i in range(n)}

    m.setObjective(quicksum(dist[i, j]*x[i, j] for (i, j) in x), "minimize")

    for i in range(n):
        m.addCons(quicksum(x[i, j] for j in range(n) if i != j) == 1)
    for j in range(n):
        m.addCons(quicksum(x[i, j] for i in range(n) if i != j) == 1)

    m.addCons(u[0] == 0)
    for i in range(1, n): m.chgVarLb(u[i], 1.0)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                m.addCons(u[i] - u[j] + n*x[i, j] <= n - 1)
    return m

In [56]:
res = solve_ip_enumerate_like(
    build_model_fn=lambda: build_tsp_mtz_euclid(n=15, seed=7),
    node_limit=50,              # small, to stop early
    time_limit=None,
    damp_root=True
)
print(res["status"], res["nodes_processed"], "open:", len(res["active_nodes"]))
print("# integer solutions seen:", len(res["integer_solutions"]))


[paramset.c:1996] ERROR: parameter <heuristics/freq> unknown
[set.c:3308] ERROR: Error <-12> in function call
[scip_param.c:496] ERROR: Error <-12> in function call
[paramset.c:2064] ERROR: parameter <heuristics/freq> unknown
[set.c:3398] ERROR: Error <-12> in function call
[scip_param.c:612] ERROR: Error <-12> in function call
[paramset.c:2132] ERROR: parameter <heuristics/freq> unknown
[set.c:3474] ERROR: Error <-12> in function call
[scip_param.c:728] ERROR: Error <-12> in function call
[paramset.c:1996] ERROR: parameter <heuristics/emphasis> unknown
[set.c:3308] ERROR: Error <-12> in function call
[scip_param.c:496] ERROR: Error <-12> in function call
[paramset.c:2064] ERROR: parameter <heuristics/emphasis> unknown
[set.c:3398] ERROR: Error <-12> in function call
[scip_param.c:612] ERROR: Error <-12> in function call
[paramset.c:2132] ERROR: parameter <heuristics/emphasis> unknown
[set.c:3474] ERROR: Error <-12> in function call
[scip_param.c:728] ERROR: Error <-12> in function cal

TypeError: NodeTracker.__init__() missing 1 required positional argument: 'model'

In [51]:
print(res["status"], res["nodes_processed"], "open:", len(res["active_nodes"]))
print("Integer solutions:", len(res["integer_solutions"]))

optimal 1 open: 0
Integer solutions: 5


In [ ]:
from pyscipopt import Model, Eventhdlr, quicksum, SCIP_EVENTTYPE

INT_TOL = 1e-6

def _safe_set_param(model, name, value):
    try:
        if hasattr(model, "hasParam") and model.hasParam(name):
            if isinstance(value, (bool, int)): model.setIntParam(name, int(value)); return
            if isinstance(value, float):        model.setRealParam(name, float(value)); return
            if isinstance(value, str):          model.setStringParam(name, value); return
        try: model.setIntParam(name, int(value)); return
        except: pass
        try: model.setRealParam(name, float(value)); return
        except: pass
        try: model.setStringParam(name, str(value)); return
        except: pass
    except:
        pass

def _is_integral_from_curr_lp(model, vars_):
    for v in vars_:
        if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
            val = model.getSolVal(None, v)
            if abs(val - round(val)) > INT_TOL:
                return False
    return True

class NodeTracker(Eventhdlr):
    def __init__(self):
        super().__init__()
        self.active = {}
        self.sols = []
        self.allvars = None

    def eventinit(self):
        for et in [
            SCIP_EVENTTYPE.NODEBRANCHED,
            SCIP_EVENTTYPE.NODEFOCUSED,
            SCIP_EVENTTYPE.LPSOLVED,
            SCIP_EVENTTYPE.NODEINFEASIBLE,
            SCIP_EVENTTYPE.NODEFEASIBLE,
            SCIP_EVENTTYPE.NODESOLVED,
            SCIP_EVENTTYPE.BESTSOLFOUND,
        ]:
            try: self.model.catchEvent(et, self)
            except: pass
        self.allvars = self.model.getVars()

    def _mark_open(self, node):
        try:
            nid = node.getNumber(); depth = self.model.getDepth()
        except: 
            return
        if nid not in self.active:
            self.active[nid] = {"status": "open", "depth": depth, "lp_bound": None, "integral": None}

    def _close(self, node, reason):
        try: nid = node.getNumber()
        except: return
        if nid in self.active: self.active[nid]["status"] = reason

    def eventexec(self, event):
        et = event.getType()

        if et == SCIP_EVENTTYPE.NODEBRANCHED:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node: self._mark_open(node)

        elif et == SCIP_EVENTTYPE.NODEFOCUSED:
            node = self.model.getCurrentNode()
            if node: self._mark_open(node)

        elif et == SCIP_EVENTTYPE.LPSOLVED:
            node = self.model.getCurrentNode()
            if not node: 
                return
            self._mark_open(node)
            try: 
                lb = self.model.getLPObjVal()
            except: 
                lb = None
            nid = node.getNumber()
            self.active[nid]["lp_bound"] = lb

            integral = _is_integral_from_curr_lp(self.model, self.allvars)
            self.active[nid]["integral"] = integral
            if integral:
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = int(round(self.model.getSolVal(None, v)))
                try: obj = self.model.getLPObjVal()
                except: obj = None
                self.sols.append({"obj": obj, "values": vals})
                self._close(node, "integer")

        elif et == SCIP_EVENTTYPE.NODEINFEASIBLE:
            node = getattr(event, "getNode", lambda: None)() or self.model.getCurrentNode()
            if node: self._close(node, "infeasible")

        elif et == SCIP_EVENTTYPE.BESTSOLFOUND:
            sol = self.model.getBestSol()
            if sol:
                try: obj = self.model.getSolObjVal(sol)
                except: obj = None
                vals = {}
                for v in self.allvars:
                    if v.vtype() in ("BINARY", "INTEGER", "IMPLINT"):
                        vals[v.name] = int(round(self.model.getSolVal(sol, v)))
                self.sols.append({"obj": obj, "values": vals})

from pyscipopt import SCIP_PARAMSETTING

def solve_ip_enumerate_like(build_model_fn, node_limit, time_limit=None, force_branch=True):
    """
    Run B&B with a node limit, log integer-feasible solutions via NodeTracker,
    and return the true active node list using model.getOpenNodes().

    Set force_branch=True to disable presolve/heuristics/separating so root doesn't close.
    """
    # build random model
    model = build_model_fn()

    # disable heuristics/presolve/separating to avoid root closing
    if force_branch:
        model.setHeuristics(SCIP_PARAMSETTING.OFF)   # disable primal heuristics
        model.setPresolve(SCIP_PARAMSETTING.OFF)     # disable presolve
        model.setSeparating(SCIP_PARAMSETTING.OFF)   # disable cut separation

    # Stop rules 
    model.setLongintParam("limits/nodes", node_limit)
    if time_limit is not None:
        model.setRealParam("limits/time", float(time_limit))

    # attach tracker
    tracker = NodeTracker()
    model.includeEventhdlr(tracker, "NodeTracker", "Log integer nodes")

    model.optimize()

    # get open nodes
    leaves, children, siblings = model.getOpenNodes()
    def _node_info(n):
        try:
            lb = n.getLowerbound()
        except Exception:
            lb = None
        return {
            "node_id": n.getNumber(),
            "parent_id": (n.getParent().getNumber() if n.getParent() else None),
            "depth": n.getDepth(),
            "lower_bound": lb,
            "is_active": n.isActive(),
        }
    active_nodes = [_node_info(n) for n in (leaves + children + siblings)]

    try:
        nodes_remaining = model.getNNodesLeft()
    except AttributeError:
        nodes_remaining = None

    out = {
        "status": model.getStatus(),
        "nodes_processed": model.getNNodes(),
        "nodes_remaining": nodes_remaining,
        "solving_time": model.getSolvingTime(),
        "integer_solutions": tracker.sols,   # keep only tracker to avoid double-counting
        "active_nodes": active_nodes,
    }

    try:
        model.free()
    except Exception:
        pass
    return out


In [73]:
res = solve_ip_enumerate_like(
    build_model_fn=lambda: build_tsp_mtz_euclid(n=15, seed=7),
    node_limit=50,              # small, to stop early
    time_limit=None,
    force_branch=True
)
print(res["status"], res["nodes_processed"], "open:", len(res["active_nodes"]))
print("# integer solutions seen:", len(res["integer_solutions"]))


presolving:
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (0 rounds: 0 fast, 0 medium, 0 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 225 variables (210 bin, 0 int, 0 impl, 15 cont) and 213 constraints
    213 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.0s|     1 |     0 |    45 |     - |  2312k |   0 | 225 | 212 | 212 |   0 |  0 |   0 |   0 | 3.297333e+02 |      --      |    Inf | unknown
  0.0s|     1 |     2 |    46 |     - |  2397k |   0 | 225 | 212 | 212 |   0 |  1 |   0 |  21 | 3.298000e+02 |      --   

In [74]:
print(res["status"], res["nodes_processed"], "open:", len(res["active_nodes"]))
print("# integer solutions seen:", len(res["integer_solutions"]))

optimal 20 open: 0
# integer solutions seen: 6


In [77]:
print(res.keys())

dict_keys(['status', 'nodes_processed', 'nodes_remaining', 'solving_time', 'integer_solutions', 'active_nodes'])


In [78]:
print(len(res['integer_solutions']))

6
